In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install tensorflow  #if the library is not installed then install by this 

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

train images loading

In [ ]:
# Create a list to store the images and labels
train_images = []
train_labels = []

# Path to the data
path = '/dir_path'

# Loop over each class directory
for i, class_name in enumerate(os.listdir(path)):
    # Loop over each image in the class directory
    for filename in os.listdir(os.path.join(path, class_name)):
        # Load the image using OpenCV
        img = cv2.imread(os.path.join(path, class_name, filename))

        # Resize the image to a fixed size
        img = cv2.resize(img, (224,224))

        # Add the image and label to the lists
        train_images.append(img)
        train_labels.append(i)

In [ ]:
# Convert the images and labels to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
# Normalize the pixel values to the range [0,1]
train_images = train_images / 255.0

loadin validation

In [ ]:
# loading validation data
# Create a list to store the validation_images and labels
val_images = []
val_labels = []

# path to validation images
val_path = '/dir_path'

# Loop over each class directory
for i, class_name in enumerate(os.listdir(val_path)):
    # Loop over each image in the class directory
    for filename in os.listdir(os.path.join(val_path, class_name)):
        # Load the image using OpenCV
        img = cv2.imread(os.path.join(val_path, class_name, filename))

        # Resize the image to a fixed size
        img = cv2.resize(img, (224,224))

        # Add the image and label to the lists
        val_images.append(img)
        val_labels.append(i)

In [ ]:
# Convert the images and labels to NumPy arrays
val_images = np.array(val_images)
val_labels = np.array(val_labels)

In [ ]:
# Normalize the pixel values to the range [0,1]
val_images = val_images / 255.0

In [ ]:
import matplotlib.pyplot as plt

# Plot a few random samples from the training set
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(train_images[i])
plt.show()

CREATING MODEL

In [ ]:
base_model = tf.keras.applications.VGG19(
 include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3), #removing 3 from(224,224,3) as grayscale imaged are now being used
    pooling=None,
    classes=4,
    )

In [ ]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

base_model = tf.keras.applications.VGG19(
include_top=False,
  weights="imagenet",
  input_tensor=None,
  input_shape=(224,224,3), #removing 3 from(224,224,3) as grayscale imaged are now being used
  pooling=None,
  classes=4,
  )
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(5, activation='softmax')(x)

# Compile the model with a categorical cross-entropy loss function and an Adam optimizer
model = Model(inputs=base_model.input, outputs=x)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

In [ ]:
histry = model.fit(train_images, train_labels, epochs=50, batch_size=8, validation_data=(val_images, val_labels))

In [ ]:
# loading TEST data
# Create a list to store the validation_images and labels
test_images = []
test_labels = []

# test data
path_test = '/test_data'

# Loop over each class directory
for i, class_name in enumerate(os.listdir(path_test)):
    # Loop over each image in the class directory
    for filename in os.listdir(os.path.join(path_test, class_name)):
        # Load the image using OpenCV
        img = cv2.imread(os.path.join(path_test, class_name, filename))

        # Resize the image to a fixed size
        img = cv2.resize(img, (224,224))

        # Add the image and label to the lists
        test_images.append(img)
        test_labels.append(i)

In [ ]:
# Convert the images and labels to NumPy arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Normalize the pixel values to the range [0,1]
test_images = test_images / 255.0


In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, batch_size=8)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

In [ ]:
print("val labels:", val_labels) #validation labels should have the labels of all 5 classes
print("Test labels:", test_labels)
print("Train labels:", train_labels) # train labels should have the labels of all 5 classes

saving model weights through this command

In [ ]:
model.save('my_model_RESNET%)_#CLASSES.h5')

In [ ]:
model.save('saved_model_RESNET50_3classes/my_model')

loading previously saved model

CLASSIFICATION REPORT

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
# Predict the classes for test data
predictions = model.predict(test_images)
#convert label array into 1D
test_labels = np.array(test_labels).ravel()
# Convert predictions to class labels
predictions = np.argmax(predictions, axis=1)
print(predictions)
# Get the class-wise accuracy
print(classification_report(test_labels, predictions))

In [ ]:
# Get the class-wise accuracy
print(classification_report(test_labels, predictions))

checking no. of images for training, testing and validation

In [ ]:
no_images_train=train_images.shape[0]
no_images_test=test_images.shape[0]
no_images_val=val_images.shape[0]
print('no. of images in training:', no_images_train)
print('no. of images in testing:', no_images_test)
print('no. of images in validation:', no_images_val)